In [ ]:

import numpy as np
import matplotlib.patches as patches
import matplotlib.pyplot as plt
import matplotlib.animation as animation

from enum import Enum, auto
from random import uniform, random
from math import cos, sin, pi, inf, sqrt
import os


radius = 0.075    # 粒子半径.
dt = 0.01         # 時間刻み.0.01
half = 1.5          # 正方形 Box の一辺の長さの半分.half = 5
speed = 1         # 速度の大きさ.

time_to_fever = 1.2      # 感染から発熱までの時間
time_to_recover = 1.5     # 感染から回復にかかる時間（回復率 γ の逆数）.time_to_recover = 2
buff = 10              # バッファ.buff = 300

fig = plt.figure(figsize=plt.figaspect(1/1))    # 図.
fig.set_facecolor('black')                      # 図の背景色は黒.


class State(Enum):
    """
    Susceptible: 未感染状態.
    Communicate: 感染源
    Infected:    感染発熱.
    Recovered:   免疫獲得状態.
    """
    Susceptible = auto()
    Communicate = auto()
    Infected = auto()
    Recovered = auto()
    
class Contact:
    
    def __init__(self):
        self.c_numbers=([0])
        
        
class Person:
    
    #_呼出は、、2回
    #_147 _infected = Person(-,-,-)
    #_180 _self.persons.append(Person(-,-,-)

    """
    人間を表すクラス
    Attributes 属性
    ----------
    co: np.ndarray       現在の 2 次元座標
    coords: np.ndarray   過去の座標を全て記憶
    arg: float           初期速度の偏角
    standstill: bool     停止しているか否か.
    vel: np.ndarray      現在の速度（次元2_x,y）
    state: State         現在の感染状態
    infection_start: int    感染が始まった時間の index
    communicate_start: int  伝えた時間の index
    """

    def __init__(self, co, state, standstill ):
        """
        (Parameters:変数)
        co: np.ndarray     212,213で[x,y]初期 2次元座標と現在座標                  
        state: State        初期感染状態
        standstill: bool    停止(True)させるか、動かす(False)か
        """
        self.co = co
        self.coords = np.array([self.co])
        
        self.arg = uniform(-pi, pi)
        self.standstill = standstill
        
        # 停止、_init_初期速度(0.0, 0.0).
        if self.standstill:
            vel = np.array([0.0, 0.0])
        else:
            vel = np.array([speed * cos(self.arg), speed * sin(self.arg)])
            
        self.vel = vel
        self.state = state
        self.infection_start = inf
        self.communicate_start = inf    #_伝えた

    def move(self):
        """
        self が、停止なら(True), 動かす.
        動いているは、vel(x,y)はそのまま、
        """
        if self.standstill:
            self.standstill = False
            self.vel = np.array([speed * cos(self.arg), speed * sin(self.arg)])
            

class Population:

    def __init__(self, number, p=None):
        """
        (Parameters:変数)
        number: int     総人口数.
        p: float        未感染者の感染確率
                        感染防止対策_マスク_3密回避_手洗いうがい消毒の実施度
        """
        
        self.number = number
        self.R0 = (radius * (sqrt(2) * speed) * time_to_recover * self.number) / (half ** 2)
        self.s_R0 = format(self.R0, '.1f')
        print("R0: %f" % self.R0,self.s_R0)
        
        self.persons = []        #_number_の座標他属性の記録列
        self.c_persons = []        #_number_の接触履歴の記録列
        
        self.p = p
        init_coords = []
        if self.p is not None:
            self.Re = (1.0 - self.p) * (1.0 + (sqrt(2) - 1) * self.p) * self.R0
            self.s_Re = format(self.Re, '.3f')
            print("Re: %f" % self.Re,self.s_Re)
            
        # 総人口数分の人間_self.persons[0],self.persons[1],self.persons[2],-,生成.
        
        for i in range(self.number):
            if i == 0:
                # 最初の人間_self.persons[0]は「感染源」Communicate_として、初期条件を設定.
                # 初期座標は原点に設定.
                init_co = np.array([0.0, 0.0])
                
                infected = Person(co = init_co, state = State.Communicate, standstill = False)
                
                #_感染が始まった時間=0
                infected.infection_start = 0
                
                self.persons.append(infected)
                
                #_number_分の_接触numberを記録する列を作成
                c_infected = Contact()
                self.c_persons.append(c_infected)
                
                
            else:
                #_self.persons[0]以外の人間は未感染者として初期条件を設定.
                while True:
                    # 現在設定中の人の初期座標_A_は、
                    # 以前の人間の初期座標_B_との隙間が_(radius_距離)となる座標を得るまで
                    # while ループでランダムに生成.
                    
                    init_co = np.array([uniform(-half, half), uniform(-half, half)])
                    appropriate = True
                    
                    for pre_init_co in init_coords:
                        dist = np.linalg.norm(init_co - pre_init_co, ord=2)

                        if dist <= 3 * radius:
                            appropriate = False
                    if appropriate:
                        break
                        
                #_number_分の_初期座標列_self.persons[0],[1],[2],[3],[4]作成し
                #_341_で、各人間に対し, 移動した現在座標_A_を追加記憶
                
                self.persons.append(Person(co=init_co, state=State.Susceptible, standstill=False))
                
                #_number_分の_接触_number_列、作成
                
                c_infected = Contact()
                self.c_persons.append(c_infected)
                #_print("_170_self.c_persons[i]=",self.c_persons[i])
                
            #_座標メモ_init_coords
            init_coords.append(init_co)
            
        #_498_Graph_で以下データ使用
        self.times = np.array([0])
        self.zeros = np.array([0])
        self.S = np.array([self.number - 1])
        self.C = np.array([1])
        self.I = np.array([0])
        self.R = np.array([0])
        
        
    def simulate_sir(self):
        
        i = 0    # 時間の index.
        
        S = (self.number-1)
        C = 1
        I = 0
        R = 0
        
        subsided = None    # 感染が収束した時の時間のインデックス
        
        while True:
            
            # 速度の時間刻み dt のスカラー倍分移動
            # 一人移動（新座標_A）させ、他の人の、旧座標_Bとの、接触を検出する
            # 正方形 Box の壁に当たった場合には反射させる
            
            for person in self.persons:
                
                if (i - person.infection_start) * dt > time_to_recover:
                    person.state = State.Recovered
                
                person.co += dt * person.vel                    # 移動 
                
                if abs(person.co[0]) > half:                      # x_軸方向での反射.
                    person.co[0] = np.sign(person.co[0]) * half
                    person.vel[0] *= -1
                
                if abs(person.co[1]) > half:                      # y_軸方向での反射.
                    person.co[1] = np.sign(person.co[1]) * half
                    person.vel[1] *= -1
                    
                
            # 各人間の接触と、2次感染を判定する.
            
            indices = list(range(self.number))
            
            while len(indices) > 0:
                k = indices.pop(0)        # [0,1,2,3]→k=0,[1,2,3]→k=1,[2,3]
                person = self.persons[k]               
                
                for l in indices:               #_[1,2,3]→[2,3]
                    other = self.persons[l]
                    vec = person.co - other.co
                    dist = np.linalg.norm(vec, ord=2)
                    
                    # person と other の距離が 2 *（半径）より小さいとき、接触したと判定.
                    # 簡単のため 3 人以上の同時接触は考えない.
                    
                    if 0 < dist < 2 * radius:
                        
                        normal = vec / dist
                        
                        # person と other が重ならないようにするため、距離を開く補正（correction）
                        
                        correction = (radius - (dist / 2)) * normal
                        person.co += correction
                        other.co -= correction                   
                        person.vel -= (2 * np.dot(person.vel, normal)) * normal     # 衝突
                        other.vel -= (2 * np.dot(other.vel, normal)) * normal      # 衝突
                        
                        indices.remove(l)                                 # [1,2,3]→[2,3]
                         
                        # 感染源[0]の、接触感染履歴列は、self.C_persons[0].c_number→[0,3,5]と記録
                        # object列_self.C_persons[0],[1],[2],-
                        # 感染対策度_self.p_は
                        #_マスク、三密回避、会話距離、手洗い、うがい、アルコール消毒をしている人が
                        #_感染源に接触時、「2次感染しない確率_self.p=0.8_」とは、「2割の確率で感染する」こと
                        
                        #__C_接触_S
                        if person.standstill == False and person.state == State.Communicate and other.state == State.Susceptible:
                            if self.p <= random():
                                other.state = State.Communicate
                                other.infection_start = i
                                C += 1
                                S -= 1
                            
                                #_感染源[-]_の、感染拡大の履歴
                                c_num = self.c_persons[k].c_numbers
                                other_number=(l)
                                c_num.append(other_number)
                                #_print(i,"_265_感染履歴_append後self.c_persons[",k,"].c_numbers=",c_num,"\n")
                            #___
                        #__S_接触_C
                        elif other.standstill == False and person.state == State.Susceptible and other.state == State.Communicate:
                            if self.p <= random():
                                person.state = State.Communicate
                                person.infection_start = i
                                C += 1
                                S -= 1
                                
                                #_感染源[-]_の、感染拡大の履歴
                                c_num = self.c_persons[l].c_numbers
                                person_number= (k)
                                c_num.append(person_number)
                                #_print(i,"_279_感染履歴append後self.c_persons[",l,"].c_numbers=",self.c_persons[l].c_numbers,"\n")
                            #___
                        break
                        
                #_for_l_in_indices_索引:
                
            num_p=0
            for p in self.persons:
                
                a1=False
                if p.state == State.Communicate:                #_26_class_State_
                    a1 = True                                   #_感染源Flag_ _256_270で、感染源に設定
                    
                a2=False
                if time_to_fever == ((i-p.infection_start)*dt):
                    a2 = True                                    #_感染源 _p_が、発熱した
                    
                a3=False
                if p.state == State.Infected:
                    a3 = True                                    #_発熱者Flag
                    
                a4=False
                if time_to_recover == ((i-p.infection_start)*dt):
                    a4 = True                                     #_回復期間、経過した
                    
                if a1 and a2:
                    p.state = State.Infected                     #_感染源が、発熱して、発熱者に_state_更新
                    p.standstill = True                          #_発熱者は、停止
                    p.vel = np.array([0.0, 0.0])
                    C -= 1
                    I += 1
                    
                    #_感染源が、fever発熱時、過去接触者を停止、自主隔離で、aura_消す_427
                    #_感染履歴リスト self.c_persons[ 0 ].c_numbers= [0, 3, 1]
                    
                    c_list = self.c_persons[num_p].c_numbers
                    
                    #__print(i,"_316_2次感染履歴_self.c_persons[",num_p,"].c_numbers=",c_list)
                    
                    for c in c_list:
                        
                        if 0 < c:
                            self.persons[c].standstill = True            #__1次感染源fever_で、2次感染源停止
                            self.persons[c].vel = np.array([0.0, 0.0])
                            self.persons[c].communicate_start = i        #_427_感染源の_aura_停止用
                            
                            
                if a3 and a4:
                    p.state = State.Recovered        #_回復期間、経過した発熱者を、回復者にして、88_def_move()で動かす
                    
                    p.move()
                    
                    I -= 1
                    R += 1
                    
                num_p += 1
                
            #_223_person = self.persons[k]
            #_240_person.co += correction   位置補正座標
            # 各人間に対し, 現在の座標を追加記憶させる.
            
            for person in self.persons:
                person.coords = np.vstack([person.coords, person.co])
                
            if C == 0 and I == 0 and subsided is None:
                subsided = i
                print(i,"_345_収束_index_subsided=",subsided,"\n")
                
            # それぞれの数値計算結果を記憶.
            self.times = np.append(self.times, i * dt)
            self.zeros = np.append(self.zeros, 0)
            
            #_200_self.S = np.array([self.number - 1])
            self.S = np.append(self.S, S)
            self.C = np.append(self.C, C)
            self.I = np.append(self.I, I)
            self.R = np.append(self.R, R)
            
            if subsided is not None and (subsided + buff) <= i:
                self.end = i
                print(i,"_359_simu-while_break_self.end=",self.end,"\n")
                break
                
            if i==10000:
                self.end = i
                print(i,"_364_simu-while_break_self.end = 10000=",self.end,"\n")
                break
                
            i += 1
            #_while True:_時間_index_i
            
    def animate(self):
        """
        SIR のシミュレーション & グラフのアニメーションを作成.
        """
        def simulate(i):
            """
            時間の index が i の時のシミュレーションフレームを描画.

            Parameters
            ----------
            i: int
                時間の index.
            """
            
            plt.cla()
            #_目盛りの数値 & 目盛りを消去.
            plt.tick_params(labelbottom=False,labelleft=False,labelright=False,
                            labeltop=False,bottom=False,left=False,right=False,top=False)
            
            ax = fig.add_subplot()
            
            ax.spines['top'].set_color('white')        #_正方形 Box の枠の色は白色で設定
            ax.spines['bottom'].set_color('white')
            ax.spines['left'].set_color('white')
            ax.spines['right'].set_color('white')
            
            ax.set_xlim(-half, half)           #_x の範囲は -half ~ half まで
            ax.set_ylim(-half, half)           #_y の範囲は -half ~ half まで
            ax.set_aspect('equal')             #_アスペクト比は等しく設定
            ax.set_facecolor('black')          #_背景色は黒で設定
            
            # 各人間を, 未感染者_lime色の円
            # 発熱前の感染者は、感染源_yellow色の円
            # 発熱治療者は_red赤色の円、感染源ではない
            # 回復者は_blue色の円、再感染しない設定
            # 時間の index が i の時の、各人間の座標の位置に、描画.
            
            for p in self.persons:
                
                if i < p.infection_start:
                    #_感染していない、未感染状態_State.Susceptible  _85_infection_start=inf
                    
                    circ = patches.Circle(xy=tuple(p.coords[i]), radius=radius, fc='lime')
                    
                elif p.infection_start <= i <= p.infection_start + time_to_fever / dt:
                    #_この時間帯は、感染源.State.Communicate
                    
                    circ = patches.Circle(xy=tuple(p.coords[i]), radius=radius, fc='yellow')
                    
                    p_aura = 10 * (i - p.infection_start) * dt / time_to_fever
                    aura_radius = 2.5 * (p_aura - int(p_aura)) * radius
                    aura = patches.Circle(xy=tuple(p.coords[i]), radius=aura_radius, ec='yellow',fill=False)
                    
                    #_323_communicate_start=i_で知らせを受け、自主隔離の人は、非感染源(オーラ_無)
                    #_知らせを受ける前の人は_aura_感染源
                    #__感染源は、オーラを周期的に描画.
                    if i <= p.communicate_start:
                        ax.add_patch(aura)
                        
                        
                elif p.infection_start+time_to_fever < i <= p.infection_start + time_to_recover / dt:
                    #_この時間帯は、発熱_State.Infected
                    
                    circ = patches.Circle(xy=tuple(p.coords[i]), radius=radius, fc='red')
                    
                else:
                    #_この時間帯は、免疫状態._State.Recovered
                    circ = patches.Circle(xy=tuple(p.coords[i]), radius=radius, fc='blue')
                    
                ax.add_patch(circ)
                
            #_
            
        simula_name = "SCIR_simu" + str(self.number)+ "_h"+ str(half)+ "_f"+ str(time_to_fever)+ "対策_p" + str(self.p)+ "_R" + str(self.s_R0)
        
        # シミュレーションのアニメーションを作成は、374_def_simulate(i)の、i_に_0～end_まで繰り返し作図
        
        animate(update=simulate, end=self.end, name=simula_name)
        
        def draw_graph(i):
            """
            時間の index が i の時のグラフフレームを描画.

            Parameters
            ----------
            i: int
                時間の index.
            """
            
            plt.cla()
            # x 軸, y 軸の目盛りの値 & 目盛りは描画.
            plt.tick_params(labelbottom=True,labelleft=True,labelright=False,
                            labeltop=False,bottom=True,left=True,right=False,top=False)
            
            ax = fig.add_subplot()
            ax.spines['top'].set_color('white')
            ax.spines['bottom'].set_color('white')
            ax.spines['left'].set_color('white')
            ax.spines['right'].set_color('white')
            
            ax.tick_params(axis='x', colors='white')    #_x軸目盛りの色は白で設定
            
            ax.tick_params(axis='y', colors='white')    #_y軸目盛りの色は白で設定
            
            ax.set_aspect('auto')                      #_アスペクト比は自動設定
            ax.set_facecolor('black')
            
            
            
            
            # 
            # 
            # 
            
            times = self.times[:i + 1]        #_時間の index が i までの時間の配列,
            zeros = self.zeros[:i + 1]        #_ゼロ配列
            
            S = self.S[:i + 1]            #_回復者数の推移の配列未感染者数
            C = self.C[:i + 1]            #_感染者で発熱前の人数の推移の配列
            I = self.I[:i + 1]            #_発熱治療者数の推移の配列
            R = self.R[:i + 1]            #_回復者数の推移の配列
            
            #_それぞれの推移を、スライスで取得描画.区間塗潰し
            ax.fill_between(times,
                            C + I + S,
                            C + I + S + R,
                            facecolor='blue',
                            alpha=1.0,
                            label=str(int(self.R[i])) + "（免疫者）")
            
            ax.fill_between(times,
                            C + I,
                            C + I + S,
                            facecolor='lime',
                            alpha=1.0,
                            label=str(int(self.S[i])) + "（未感染者)"+ "_p" + str(self.p))
            
            ax.fill_between(times,
                            C,
                            C+I,
                            facecolor='red',
                            alpha=1.0,
                            label=str(int(self.I[i])) + "（発熱患者）")
            
            ax.fill_between(times,
                            zeros,
                            C,
                            facecolor='yellow',
                            alpha=1.0,
                            label=str(int(self.I[i])) + "（発熱前感染者）")
            
            ax.legend(loc="upper left", prop={"family": "MS Gothic"})
            
        graph_name = "SCIR_graph" + str(self.number)+ "_h"+ str(half)+ "_f"+ str(time_to_fever)+ "対策_p" + str(self.p)+ "_R" + str(self.s_R0)
        
        # グラフの推移アニメーションを作成.は、453_def_draw_graph(i)の、i_に_0～end_まで繰り返し作図
        
        animate(update=draw_graph, end=self.end, name=graph_name)
        
        
def animate(update, end, name):
    """
    matplotlib.animation.FuncAnimation によるアニメーション作成.
    ffmpeg をインストールしておく必要がある.
    Parameters
    ----------
    update: func    各フレームを描画する関数を指定.
    end: int    終了フレーム.
    name: str    アニメーション保存時の名前.
    """
    anim = animation.FuncAnimation(fig, update, frames=end, interval=100/3)
    
    anim.save(name + ".mp4", writer='ffmpeg', dpi=300, savefig_kwargs={'facecolor':'black'})
    
    
if __name__ == "__main__":
    
    population = Population(number=100,p=0.8)    #_感染しない確率p=0.8_の感染_254_268
    #                                            #_感染しない確率p=0.2_に書換_simulate
    population.simulate_sir()
    
    population.animate()
    

#
#